In [9]:
%load_ext autoreload
%autoreload 2
import glob
import json

outs = {}


for filename in sorted([f for f in glob.glob("../output/*.json") if "test" not in f]): 
    model_name = os.path.basename(filename).split(".")[0]
    print(model_name)
    with open(filename) as f:
        outs[model_name] = json.load(f)


[autoreload of _virtualenv failed: Traceback (most recent call last):
  File "/home/jmperez/.cache/pypoetry/virtualenvs/finetune-vs-scratch-gHiQbun3-py3.8/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jmperez/.cache/pypoetry/virtualenvs/finetune-vs-scratch-gHiQbun3-py3.8/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/jmperez/.pyenv/versions/3.8.5/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/jmperez/.pyenv/versions/3.8.5/lib/python3.8/importlib/__init__.py", line 166, in reload
    spec = module.__spec__ = _bootstrap._find_spec(name, pkgpath, target)
  File "<frozen importlib._bootstrap>", line 914, in _find_spec
  File "/home/jmperez/.cache/pypoetry/virtualenvs/finetune-vs-scratch-gHiQbun3-py3.8/lib/python3.8/site-packages/_virtualenv.py", line 55, in find_spec
  

In [7]:
import pandas as pd
import numpy as np

resume = []

task_metrics = {
    "context_hate": ["eval_mean_f1", "eval_hate_f1"],
    "sentiment": ["eval_macro_f1", "eval_micro_f1"],
    "emotion": ["eval_macro_f1", "eval_micro_f1"],
}

    
for model_name, output in outs.items():
    line = {
        "model": model_name, 
    }

    for task, metrics in task_metrics.items():
        for metric in metrics:
            arr = np.array([evaluation[metric] for evaluation in output[task]])
            metric_name = metric.replace("eval_", "")
            line[task+" "+metric_name] = f"{arr.mean():.4f} +- {arr.std():.3f}"
    resume.append(line)

df = pd.DataFrame(resume)
df.set_index("model", inplace=True)
df.loc[[
    "bertin",
    "beto-cased",
    "beto-uncased",
    "beto-ft-1000",
    "beto-ft-2000",
    "beto-uncased-1000",
    "beto-uncased-2000",
]]

,context_hate mean_f1,context_hate hate_f1,sentiment macro_f1,sentiment micro_f1,emotion macro_f1,emotion micro_f1
model,,,,,,
bertin,0.5443 +- 0.005,0.6613 +- 0.009,0.6650 +- 0.002,0.6756 +- 0.003,0.5245 +- 0.026,0.6569 +- 0.019
beto-cased,0.5870 +- 0.007,0.6884 +- 0.005,0.6617 +- 0.005,0.6673 +- 0.005,0.5246 +- 0.016,0.6794 +- 0.015
beto-ft-1000,0.5857 +- 0.003,0.6861 +- 0.002,0.6721 +- 0.001,0.6786 +- 0.004,0.5335 +- 0.012,0.6798 +- 0.020
beto-ft-2000,0.5848 +- 0.005,0.6865 +- 0.003,0.6751 +- 0.005,0.6789 +- 0.007,0.5257 +- 0.010,0.6957 +- 0.006
beto-uncased-1000,0.5950 +- 0.003,0.6934 +- 0.002,0.6717 +- 0.005,0.6810 +- 0.011,0.5364 +- 0.009,0.6989 +- 0.006
beto-uncased-2000,0.5989 +- 0.004,0.6954 +- 0.002,0.6812 +- 0.001,0.6884 +- 0.005,0.5354 +- 0.005,0.6887 +- 0.016
beto-uncased,0.5906 +- 0.004,0.6856 +- 0.006,0.6517 +- 0.002,0.6573 +- 0.003,0.5250 +- 0.014,0.6681 +- 0.016
